In [7]:
# Geographic Crosswalk Creator for Japanese Administrative Boundaries
#
# This notebook creates geographic crosswalks between different years of Japanese administrative 
# boundaries, implementing a spatial standardization procedure using 2000 boundaries as the 
# reference unit. Following Eckert et al. (2020), it constructs precise geographic crosswalks
# by intersecting historical district boundaries with the reference map.
#
# Author: Shizuka  Inoue
# Date: 2025/02/24

import os
import pandas as pd
import geopandas as gpd
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Set up directory paths
notebook_dir = Path(os.path.dirname(os.path.abspath('__file__')))
project_root = notebook_dir.parent
base_dir = project_root / 'Data'
export_dir = project_root / 'Crosswalk'

# Validate directories
if not base_dir.exists():
    raise FileNotFoundError(f"Data directory not found at: {base_dir}")
export_dir.mkdir(parents=True, exist_ok=True)

logging.info(f"Using data directory: {base_dir}")
logging.info(f"Using export directory: {export_dir}")

# Configuration parameters
REFERENCE_YEAR = 2000  # Base year for standardization
THRESHOLD = 0.0001     # Minimum weight threshold for considering area overlaps
TARGET_YEARS = [1980]  # Years to create crosswalks for

def process_shapefile(file_path: Path, year: str) -> gpd.GeoDataFrame:
    """
    Load and process a shapefile for analysis.
    
    Args:
        file_path: Path to the shapefile
        year: Year identifier for the data
        
    Returns:
        Processed GeoDataFrame with dissolved boundaries and calculated areas
    """
    # Load shapefile with proper encoding
    gdf = gpd.read_file(file_path, encoding='shift_jis')
    
    # Dissolve multiple areas in one city to city level using N03_007 (City Code)
    gdf['dissolve_key'] = gdf['N03_007']
    gdf = gdf.dissolve(by="dissolve_key").reset_index()
    
    # Convert to projected CRS for area calculations in square meters
    gdf = gdf.to_crs({'proj':'cea'})
    gdf['geometry'] = gdf['geometry'].buffer(0)
    gdf[f'Area ({year})'] = gdf.area/10**6
    
    return gdf

def create_crosswalk(base_gdf: gpd.GeoDataFrame, target_year: int) -> gpd.GeoDataFrame:
    """
    Create geographic crosswalk between reference year and target year.
    
    Args:
        base_gdf: GeoDataFrame for reference year
        target_year: Year to create crosswalk with
        
    Returns:
        GeoDataFrame containing the crosswalk
    """
    # Load target year data
    target_path = base_dir / f'jpn{target_year}' / f'jpn{target_year}geo.shp'
    target_gdf = process_shapefile(target_path, str(target_year))
    
    # Verify coordinate systems match
    logging.info("Checking spatial alignment:")
    logging.info(f"Reference bounds: {base_gdf.total_bounds}")
    logging.info(f"Target bounds: {target_gdf.total_bounds}")
    
    # Perform spatial overlay
    try:
        intersect = gpd.overlay(base_gdf, target_gdf, how='intersection', keep_geom_type=False)
        logging.info(f"Spatial overlay successful for year {target_year}")
    except Exception as e:
        logging.error(f"Overlay failed: {str(e)}")
        try:
            intersect = gpd.overlay(base_gdf, target_gdf, how='intersection', 
                                  keep_geom_type=False, use_sindex=False)
            logging.info("Overlay successful using alternative method")
        except Exception as e2:
            logging.error(f"Alternative overlay also failed: {str(e2)}")
            raise
    
    return intersect

def calculate_weights(intersect: gpd.GeoDataFrame, year: int) -> gpd.GeoDataFrame:
    """
    Calculate and normalize weights for the crosswalk.
    
    Args:
        intersect: GeoDataFrame with intersection results
        year: Target year
        
    Returns:
        GeoDataFrame with calculated weights
    """
    # Calculate intersection areas and weights
    intersect['intersect_area'] = intersect.geometry.area/10**6
    intersect['weight'] = intersect['intersect_area'] / intersect[f"Area ({year})"]
    
    # Log weight statistics
    weight_sums = intersect.groupby(['PREF_2', 'GUN_2', 'CITY_2'])['weight'].sum()
    logging.info(f"\nWeight statistics for {year}:")
    logging.info(f"Mean weight sum: {weight_sums.mean():.3f}")
    logging.info(f"Min weight sum: {weight_sums.min():.3f}")
    logging.info(f"Max weight sum: {weight_sums.max():.3f}")
    logging.info(f"Areas with weights < 0.95: {sum(weight_sums < 0.95)}")
    
    # Filter small weights and normalize
    intersect = intersect[intersect['weight'] > THRESHOLD]
    intersect['weight'] = intersect.groupby(['PREF_2', 'GUN_2', 'CITY_2'])['weight'].transform(
        lambda x: x / x.sum()
    )
    
    return intersect

def format_output(intersect: gpd.GeoDataFrame, target_year: int) -> pd.DataFrame:
    """
    Format the final crosswalk output.
    
    Args:
        intersect: GeoDataFrame with calculated weights
        target_year: Target year
        
    Returns:
        DataFrame with formatted crosswalk data
    """
    # Rename columns for consistency
    rename_dict = {
        'PREF_1': f'PREF{REFERENCE_YEAR}',
        'PREF_2': f'PREF{target_year}',
        'CITY_1': f'CITY{REFERENCE_YEAR}',
        'CITY_2': f'CITY{target_year}',
        'GUN_1': f'GUN{REFERENCE_YEAR}',
        'GUN_2': f'GUN{target_year}',
        'N03_007_1': f'City Code {REFERENCE_YEAR}',
        'N03_007_2': f'City Code {target_year}'
    }
    intersect = intersect.rename(columns=rename_dict)
    
    # Select final columns
    keep_columns = [
        f'CITY{REFERENCE_YEAR}', f'CITY{target_year}',
        f'PREF{REFERENCE_YEAR}', f'PREF{target_year}',
        f'GUN{REFERENCE_YEAR}', f'GUN{target_year}',
        f'City Code {REFERENCE_YEAR}', f'City Code {target_year}',
        'weight'
    ]
    
    return intersect[keep_columns].drop_duplicates()

# Main execution
logging.info("Starting crosswalk creation process")

# Load reference year data
base_path = base_dir / f'jpn{REFERENCE_YEAR}' / f'jpn{REFERENCE_YEAR}geo.shp'
base_gdf = process_shapefile(base_path, str(REFERENCE_YEAR))

# Process each target year
for year in TARGET_YEARS:
    logging.info(f"\nProcessing year {year}")
    
    try:
        # Create crosswalk
        intersect = create_crosswalk(base_gdf, year)
        
        # Calculate weights
        intersect = calculate_weights(intersect, year)
        
        # Format and export
        final_df = format_output(intersect, year)
        export_path = export_dir / f'Crosswalk_{REFERENCE_YEAR}_{year}.xlsx'
        final_df.to_excel(export_path, index=False)
        
        logging.info(f"Successfully exported crosswalk to {export_path}")
        
    except Exception as e:
        logging.error(f"Failed to process year {year}: {str(e)}")
        continue

logging.info("Crosswalk creation process complete")

2025-02-24 20:10:51,970 - INFO - Using data directory: /Users/shizukainoue/Desktop/Research/Crosswalk/Data
2025-02-24 20:10:51,971 - INFO - Using export directory: /Users/shizukainoue/Desktop/Research/Crosswalk/Crosswalk
2025-02-24 20:10:51,971 - INFO - Starting crosswalk creation process
2025-02-24 20:10:54,071 - INFO - 
Processing year 1980
2025-02-24 20:10:56,327 - INFO - Checking spatial alignment:
2025-02-24 20:10:56,339 - INFO - Reference bounds: [13684894.6214349   2583224.77590051 17141256.5835307   4533543.53047737]
2025-02-24 20:10:56,355 - INFO - Target bounds: [13684894.6214349   2583224.77590051 17141256.5835307   4533543.53047737]
2025-02-24 20:11:27,105 - INFO - Spatial overlay successful for year 1980
2025-02-24 20:11:27,212 - INFO - 
Weight statistics for 1980:
2025-02-24 20:11:27,212 - INFO - Mean weight sum: 1.000
2025-02-24 20:11:27,212 - INFO - Min weight sum: 0.962
2025-02-24 20:11:27,213 - INFO - Max weight sum: 1.000
2025-02-24 20:11:27,213 - INFO - Areas with w